In [ ]:
import os
import xml.etree.ElementTree as ET

input_folder1 = r"C:\Users\tangl\Desktop\output\test"
input_folder2 = r"C:\Users\tangl\Desktop\output\ocr_change"
output_folder = r"C:\Users\tangl\Desktop\output\final_inout"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 遍历第一个文件夹中的所有XML文件
for file_name1 in os.listdir(input_folder1):
    if file_name1.endswith('.xml'):
        # 解析第一个XML文件
        tree1 = ET.parse(os.path.join(input_folder1, file_name1))
        root1 = tree1.getroot()
        
        # 遍历第二个文件夹中的同名XML文件
        file_name2 = os.path.join(input_folder2, file_name1)
        if os.path.exists(file_name2):
            tree2 = ET.parse(file_name2)
            root2 = tree2.getroot()
            
            # 遍历第二个XML文件中的所有object
            for object2 in root2.findall('object'):
                # 获取object2的bndbox
                bndbox2 = object2.find('bndbox')
                xmin2 = int(bndbox2.find('xmin').text)
                ymin2 = int(bndbox2.find('ymin').text)
                xmax2 = int(bndbox2.find('xmax').text)
                ymax2 = int(bndbox2.find('ymax').text)
                
                # 遍历第一个XML文件中的所有object
                for object1 in root1.findall('object'):
                    # 获取object1的bndbox
                    bndbox1 = object1.find('bndbox')
                    xmin1 = int(bndbox1.find('xmin').text)
                    ymin1 = int(bndbox1.find('ymin').text)
                    xmax1 = int(bndbox1.find('xmax').text)
                    ymax1 = int(bndbox1.find('ymax').text)
                    
                    # 判断object2的bndbox是否在object1的bndbox范围内
                    if xmin2 >= xmin1 and ymin2 >= ymin1 and xmax2 <= xmax1 and ymax2 <= ymax1:
                        # 将object2的name添加到object1的name中
                        name1 = object1.find('name')
                        name2 = object2.find('name').text
                        if name2 not in name1.text:
                            name1.text = name1.text + ',' + name2
                        
        # 写入新的XML文件
        tree1.write(os.path.join(output_folder, file_name1))


In [ ]:
import os
import xml.etree.ElementTree as ET
from math import sqrt

input_folder1 = r"C:\Users\tangl\Desktop\output\test"
input_folder2 = r"C:\Users\tangl\Desktop\output\ocr_change"
output_folder = r"C:\Users\tangl\Desktop\output\final_both"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def calc_distance(x1, y1, x2, y2):
    # 计算两个点之间的欧几里得距离
    return sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

# 遍历第一个文件夹中的所有XML文件
for file_name1 in os.listdir(input_folder1):
    if file_name1.endswith('.xml'):
        # 解析第一个XML文件
        tree1 = ET.parse(os.path.join(input_folder1, file_name1))
        root1 = tree1.getroot()
        
        # 遍历第二个文件夹中的同名XML文件
        file_name2 = os.path.join(input_folder2, file_name1)
        if os.path.exists(file_name2):
            tree2 = ET.parse(file_name2)
            root2 = tree2.getroot()
            
            # 遍历第二个XML文件中的所有object
            for object2 in root2.findall('object'):
                # 获取object2的bndbox
                bndbox2 = object2.find('bndbox')
                xmin2 = int(bndbox2.find('xmin').text)
                ymin2 = int(bndbox2.find('ymin').text)
                xmax2 = int(bndbox2.find('xmax').text)
                ymax2 = int(bndbox2.find('ymax').text)
                
                xcenter2 = (xmin2 + xmax2) / 2
                ycenter2 = (ymin2 + ymax2) / 2

                found_overlap = False
                
                # 遍历第一个XML文件中的所有object
                for object1 in root1.findall('object'):
                    # 获取object1的bndbox
                    bndbox1 = object1.find('bndbox')
                    xmin1 = int(bndbox1.find('xmin').text)
                    ymin1 = int(bndbox1.find('ymin').text)
                    xmax1 = int(bndbox1.find('xmax').text)
                    ymax1 = int(bndbox1.find('ymax').text)
                    
                    # 判断object2的bndbox是否在object1的bndbox范围内
                    if xmin2 >= xmin1 and ymin2 >= ymin1 and xmax2 <= xmax1 and ymax2 <= ymax1:
                        # 将object2的name添加到object1的name中
                        name1 = object1.find('name')
                        name2 = object2.find('name').text
                        if name2 not in name1.text:
                            name1.text = name1.text + ',' + name2
                            found_overlap = True
                            break
                    
                if not found_overlap:
                    min_dist = float('inf')
                    nearest_obj = None

                    for object1 in root1.findall('object'):
                        # 获取object1的bndbox
                        bndbox1 = object1.find('bndbox')
                        xmin1 = int(bndbox1.find('xmin').text)
                        ymin1 = int(bndbox1.find('ymin').text)
                        xmax1 = int(bndbox1.find('xmax').text)
                        ymax1 = int(bndbox1.find('ymax').text)
                    
                        xcenter1 = (xmin1 + xmax1) / 2
                        ycenter1 = (ymin1 + ymax1) / 2

                        dist = calc_distance(xcenter1, ycenter1, xcenter2, ycenter2)
                        if dist < min_dist:
                            min_dist = dist
                            nearest_obj = object1

                    if nearest_obj is not None:
                        name1 = nearest_obj.find('name')
                        name2 = object2.find('name').text
                        if name2 not in name1.text:
                            name1.text = name1.text + ',' + name2
                            found_overlap = True

                # 保存处理后的XML文件到输出文件夹
                if found_overlap:
                    tree1.write(os.path.join(output_folder, file_name1), encoding='utf-8')

In [ ]:
import os
import xml.etree.ElementTree as ET

def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root

def merge_xml(xml1, xml2):
    # find all objects in xml1
    objs1 = xml1.findall('object')
    for obj1 in objs1:
        # find the bndbox element in obj1
        bndbox1 = obj1.find('bndbox')
        xmin1 = int(bndbox1.find('xmin').text)
        ymin1 = int(bndbox1.find('ymin').text)
        xmax1 = int(bndbox1.find('xmax').text)
        ymax1 = int(bndbox1.find('ymax').text)
        # find all objects in xml2
        objs2 = xml2.findall('object')
        for obj2 in objs2:
            # find the bndbox element in obj2
            bndbox2 = obj2.find('bndbox')
            xmin2 = int(bndbox2.find('xmin').text)
            ymin2 = int(bndbox2.find('ymin').text)
            xmax2 = int(bndbox2.find('xmax').text)
            ymax2 = int(bndbox2.find('ymax').text)
            # compare the bndbox coordinates
            if xmin1 == xmin2 and ymin1==ymin2 and xmax1==xmax2 and ymax1==ymax2:
                # combine the text in name elements
                name1 = obj1.find('name')
                name2 = obj2.find('name')
                for text in name2.text.split(','):
                    if text not in name1.text:
                        name1.text += ',' + text
                # remove the matched object from xml2
                xml2.remove(obj2)
    # append the remaining objects in xml2 to xml1
    for obj2 in xml2.findall('object'):
        xml1.append(obj2)
    return xml1

# set the paths of the two directories
dir1_path = r"C:\Users\tangl\Desktop\output\final_both"
dir2_path = r"C:\Users\tangl\Desktop\output\final_inout"

# set the output directory path
output_dir_path = r"C:\Users\tangl\Desktop\output\final_all"

# create the output directory if it does not exist
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path)

# iterate over the files in dir1
for file_name in os.listdir(dir1_path):
    if file_name.endswith('.xml'):
        # check if the corresponding file exists in dir2
        file_path1 = os.path.join(dir1_path, file_name)
        file_path2 = os.path.join(dir2_path, file_name)
        if os.path.exists(file_path2):
            # parse the XML files
            xml1 = parse_xml(file_path1)
            xml2 = parse_xml(file_path2)
            # merge the two XML files
            merged_xml = merge_xml(xml1, xml2)
            # write the merged XML file
            merged_xml_path = os.path.join(output_dir_path, file_name)
            ET.ElementTree(merged_xml).write(merged_xml_path)

                
#合并faster-rcnn和ocr

In [ ]:
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom
import os

def process_xml_file(input_file, output_file):
    # 解析原始XML文件
    tree = ET.parse(input_file)
    root = tree.getroot()

    # 创建新的根元素
    new_root = ET.Element('annotation')

    # 遍历每个object元素
    for obj in root.findall('object'):
        name = obj.find('name').text

        # 如果name中包含"arrow"的object（记为B）
        if 'arrow' in name or 'correlation' in name:
            # 获取B的坐标范围
            b_box = obj.find('bndbox')
            b_xmin = int(b_box.find('xmin').text)
            b_ymin = int(b_box.find('ymin').text)
            b_xmax = int(b_box.find('xmax').text)
            b_ymax = int(b_box.find('ymax').text)
            #print(b_xmin)

            # 初始化A和C
            a = None
            c = None
            a1 = None
            c1 = None

            for sibling in root.findall('object'):
                sibling_name = sibling.find('name').text
                sibling_box = sibling.find('bndbox')
                sibling_xmin = int(sibling_box.find('xmin').text)
                sibling_ymin = int(sibling_box.find('ymin').text)
                sibling_xmax = int(sibling_box.find('xmax').text)
                sibling_ymax = int(sibling_box.find('ymax').text)

                if ('variable' in sibling_name or 'variables' in sibling_name) and \
                        (sibling_xmin <= b_xmin <= sibling_xmax and sibling_ymin <= b_ymin <= sibling_ymax):
                    a = sibling

                if ('variable' in sibling_name or 'variables' in sibling_name) and \
                        (sibling_xmin <= b_xmax <= sibling_xmax and sibling_ymin <= b_ymax <= sibling_ymax):
                    c = sibling

                if ('variable' in sibling_name or 'variables' in sibling_name) and \
                        (sibling_xmin <= b_xmin <= sibling_xmax and sibling_ymin <= b_ymax <= sibling_ymax):
                    a1 = sibling
                if ('variable' in sibling_name or 'variables' in sibling_name) and \
                        (sibling_xmin <= b_xmax <= sibling_xmax and sibling_ymin <= b_ymin <= sibling_ymax):
                    c1 = sibling

            if a is not None and c is not None:
                # 创建新的object，并将A、B、C的name添加到其中
                new_obj = ET.SubElement(new_root, 'object')
                new_a_name = ET.SubElement(new_obj, 'name')
                new_a_name.text = a.find('name').text
                new_b_name = ET.SubElement(new_obj, 'name')
                new_b_name.text = name
                new_c_name = ET.SubElement(new_obj, 'name')
                new_c_name.text = c.find('name').text

            if a1 is not None and c1 is not None:
                # 创建新的object，并将A、B、C的name添加到其中
                new_obj = ET.SubElement(new_root, 'object')
                new_a_name = ET.SubElement(new_obj, 'name')
                new_a_name.text = a1.find('name').text
                new_b_name = ET.SubElement(new_obj, 'name')
                new_b_name.text = name
                new_c_name = ET.SubElement(new_obj, 'name')
                new_c_name.text = c1.find('name').text

    # 创建新的XML文件
    new_tree = ET.ElementTree(new_root)
    xml_string = minidom.parseString(ET.tostring(new_root)).toprettyxml(indent="\t")
    
    with open(output_file, "w") as file:
        file.write(xml_string)
    
# 输入文件夹路径
input_folder = r"C:\Users\tangl\Desktop\output\final_all"
# 输出文件夹路径
output_folder = r"C:\Users\tangl\Desktop\output\remix"

# 处理每个XML文件
for filename in os.listdir(input_folder):
    input_file = os.path.join(input_folder, filename)
    
    # 检查是文件再处理
    if os.path.isfile(input_file) and filename.endswith(".xml"):
        output_file = os.path.join(output_folder, filename)
        process_xml_file(input_file, output_file)

In [ ]:
import os
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

def process_xml_file(input_file, output_file):
    # 解析原始XML文件
    tree = ET.parse(input_file)
    root = tree.getroot()

    # 创建新的根元素
    new_root = ET.Element('annotation')

    # 用一个集合来跟踪已经处理过的object内容
    processed_objects = set()

    # 遍历每个object元素
    for obj in root.findall('object'):
        # 将object内容转换成字符串，以便进行集合比较
        obj_str = ET.tostring(obj, encoding='unicode')

        # 如果该object内容已经处理过，就跳过该object
        if obj_str in processed_objects:
            continue

        # 将该object内容添加到已处理集合中
        processed_objects.add(obj_str)

        # 将该object添加到新的XML中
        new_root.append(obj)

    # 创建新的XML文件
    new_tree = ET.ElementTree(new_root)
    xml_string = minidom.parseString(ET.tostring(new_root)).toprettyxml(indent="\t")
    with open(output_file, "w") as file:
        file.write(xml_string)

# 输入文件夹路径
input_folder = r"C:\Users\tangl\Desktop\output\remix"
# 输出文件夹路径
output_folder = r"C:\Users\tangl\Desktop\output\remix_new"

# 处理每个XML文件
for filename in os.listdir(input_folder):
    input_file = os.path.join(input_folder, filename)
    
    # 检查是文件再处理
    if os.path.isfile(input_file) and filename.endswith(".xml"):
        output_file = os.path.join(output_folder, filename)
        process_xml_file(input_file, output_file)

#更改xml格式，变成实体-箭头-实体

In [ ]:
import os
import xml.etree.ElementTree as ET

def format_xml_in_folder(folder_path):
    # 遍历文件夹中的所有文件
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".xml"):
            xml_file = os.path.join(folder_path, file_name)
            format_xml(xml_file)

def format_xml(xml_file):
    # 解析XML文件
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # 获取文件名并去除扩展名
    file_name = os.path.splitext(os.path.basename(xml_file))[0]

    # 创建并添加id标签
    id_element = ET.Element('id')
    id_element.text = file_name
    root.insert(0, id_element)  # 将id标签插入到根元素的第一个位置

    # 保存修改后的XML文件
    new_xml_file = xml_file  # 修改后的文件名与原文件名相同，覆盖原文件
    indent(root)  # 添加缩进，使得XML文件更易读
    tree.write(new_xml_file, encoding="utf-8", xml_declaration=True)

def indent(elem, level=0):
    # 添加缩进，用于更好地格式化XML文件
    i = "\n" + level * "  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level + 1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

# 文件夹路径
folder_path = r"C:\Users\tangl\Desktop\output\remix_new"

# 在文件夹中的所有XML文件中添加id标签并格式化
format_xml_in_folder(folder_path)

#给三元组加上id

In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

# 读取Excel文件
excel_file = r"C:\Users\tangl\Desktop\output\images.xlsx"  # 替换为实际的Excel文件名
df = pd.read_excel(excel_file)

# XML文件夹路径
xml_folder = r"C:\Users\tangl\Desktop\output\remix_new"  # 替换为实际的XML文件夹路径

# 遍历Excel表格，并将'textization'内容添加到相应的XML文件中
for index, row in df.iterrows():
    image_name = row['image']
    textization = row['textization']

    # 获取对应的XML文件路径
    xml_file = os.path.join(xml_folder, f"{image_name}.xml")

    # 解析XML文件
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # 创建并添加'textization'标签
    textization_element = ET.Element('text')
    textization_element.text = textization
    root.append(textization_element)

    # 保存修改后的XML文件
    new_xml_file = xml_file  # 修改后的文件名与原文件名相同，覆盖原文件
    indent(root)  # 添加缩进，使得XML文件更易读
    tree.write(new_xml_file, encoding="utf-8", xml_declaration=True)

def indent(elem, level=0):
    # 添加缩进，用于更好地格式化XML文件
    i = "\n" + level * "  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level + 1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i
            
#把目标文本加进去

In [ ]:
import os
import xmltodict
import json

def remove_annotation(xml_data):
    # Convert XML to dictionary
    data_dict = xmltodict.parse(xml_data)

    # Remove the "annotation" key
    if "annotation" in data_dict:
        data_dict = data_dict["annotation"]

    return data_dict

def convert_xml_to_json(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.xml'):
            input_file_path = os.path.join(input_folder, file_name)
            output_file_path = os.path.join(output_folder, file_name.replace('.xml', '.json'))

            with open(input_file_path, 'r', encoding='utf-8') as file:
                xml_data = file.read()

            json_data = remove_annotation(xml_data)

            with open(output_file_path, 'w', encoding='utf-8') as json_file:
                json.dump(json_data, json_file, indent=2)

# 输入文件夹和输出文件夹路径
input_folder_path = r"C:\Users\tangl\Desktop\output\remix_new"
output_folder_path = r"C:\Users\tangl\Desktop\output\remix_json"

# 调用函数进行转换
convert_xml_to_json(input_folder_path, output_folder_path)
#将xml变成json然后合并


In [ ]:
import os
import json

def merge_json_files(input_folder, output_file):
    merged_data = []

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.json'):
            file_path = os.path.join(input_folder, file_name)

            with open(file_path, 'r', encoding='utf-8') as json_file:
                json_data = json.load(json_file)

            merged_data.append(json_data)

    with open(output_file, 'w', encoding='utf-8') as merged_json_file:
        json.dump(merged_data, merged_json_file, indent=2)

# 输入文件夹和输出文件路径
input_folder_path = r"C:\Users\tangl\Desktop\output\remix_json"
output_file_path = r"C:\Users\tangl\Desktop\output\remix_json\remix_json.json"

# 调用函数进行合并
merge_json_files(input_folder_path, output_file_path)


In [ ]:
import json
import random

def split_dataset(input_file, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    with open(input_file, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    # 随机打乱数据
    random.shuffle(data)

    # 划分数据集
    total_samples = len(data)
    train_samples = int(train_ratio * total_samples)
    val_samples = int(val_ratio * total_samples)
    test_samples = total_samples - train_samples - val_samples

    train_data = data[:train_samples]
    val_data = data[train_samples:train_samples + val_samples]
    test_data = data[train_samples + val_samples:]

    # 保存划分后的数据集
    output_folder = os.path.dirname(input_file)
    train_file = os.path.join(output_folder, 'train.json')
    val_file = os.path.join(output_folder, 'val.json')
    test_file = os.path.join(output_folder, 'test.json')

    with open(train_file, 'w', encoding='utf-8') as train_json:
        json.dump(train_data, train_json, indent=2)

    with open(val_file, 'w', encoding='utf-8') as val_json:
        json.dump(val_data, val_json, indent=2)

    with open(test_file, 'w', encoding='utf-8') as test_json:
        json.dump(test_data, test_json, indent=2)

# 输入合并后的JSON文件和划分比例
input_json_file = r"C:\Users\tangl\Desktop\output\remix_json\remix_json.json"
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# 调用函数进行划分
split_dataset(input_json_file, train_ratio, val_ratio, test_ratio)
